In [ ]:
# 进入创建好的工程目录后， 命令行输入
django-admin startproject 项目名

In [ ]:
# 进入创建好的项目目录，创建App
python manage.py startapp App名

In [ ]:
# settings内设置注册app，加入App名
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'Test_App'
]

In [ ]:
# 设置访问HOST
ALLOWED_HOSTS = ['*',]

In [ ]:
# App-models.py内写入表结构（ORM）
from django.db import models
class Department(models.Model):
    ''' 部门表 '''
    title = models.CharField(verbose_name='标题', max_length=32)


class UserInfo(models.Model):
    ''' 员工表 '''
    name = models.CharField(verbose_name='姓名', max_length=16)
    password = models.CharField(verbose_name='密码', max_length=64)
    age = models.IntegerField(verbose_name='年龄')
    # 总长度为10位，小数位占2
    account = models.DecimalField(
        verbose_name='账户余额', max_digits=10, decimal_places=2, default=0)
    creat_time = models.DateTimeField(verbose_name='入职时间')
    # 无约束
    depart_id = models.IntegerField(verbose_name='部门id')
    # 有约束
    #   - to, 与哪张表关联（外键）
    #   - to_field, 与表中哪一列关联
    # 在django中
    #   - ForeignKey的变量定义，在生成数据列时，自动增加“_id”
    # 如果部门被删除，
    #   - 用户也删除：级联删除
    depart = models.ForeignKey(
        to='Department', to_field='id', on_delete=models.CASCADE)
    #   - 置空，但是要确保该字段可以为空
    # depart = models.ForeignKey(
    #     to='Department', to_field='id', on_delete=models.SET_NULL, null=True, blank=True)

    # 在Django中做的约束
    gender_choices = (
        (1, "男"),
        (2, "女")
    )
    gender = models.SmallIntegerField(
        verbose_name="性别", choices=gender_choices)


In [ ]:
# 链接mysql数据库, 使用Django内置数据库可以不设置此项
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'NAME': 'xxxx', # 数据库名字
        'USER': 'root',
        'PASSWORD': 'root123',
        'HOST': '127.0.0.1', # 存放数据库的机器
        'PORT': 3306
    }
}

In [ ]:
# Django 命令生成数据表
python manage.py makemigrations
python manage.py migrate

In [ ]:
# 使用django后台看到数据库，需要注册
# - App-admin.py
from App.models import *
admin.site.register(UserInfo)
admin.site.register(Department)

In [ ]:
# 在App文件夹内，创建static和templates文件夹
# +---css
# +---img
# +---js
#        jquery-3.6.0.min.js
# 
# \---plugins
#     ---bootstrap-3.4.1-dist

In [ ]:
# 模板继承
# 1. 模板定义： layout.html
{% load static %}
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
    <link rel="stylesheet" href="{% static 'plugins/bootstrap-3.4.1-dist/css/bootstrap.min.css' %}">

</head>
<body>
    <div>开头</div>

    <div>
        {% block content %}
        
        {% endblock %}
    </div>

    <div>结尾</div>

    <script src="{% static 'js/jquery-3.6.0.min.js' %}"></script>
    <script src="{% static 'plugins/bootstrap-3.4.1-dist/js/bootstrap.min.js' %}"></script>
</body>
</html>

# 2. 子模板继承
{% extends 'layout.html' %}
{% block content %}
<div>
    内容
</div>
{% endblock %}

# 3. 子页面需要引入特定的，专属的内容时，可在母模板中加入多个block
{% block css %}{% endblock %}

{% block js %}{% endblock %}


In [ ]:
# 字符串 -> datatime类型
from datetime import datetime
text = "2021-11-11"
res = datetime.strptime(text, "%Y-%m-%d")
print(res, type(res))

In [ ]:
# datetime类型 -> 字符串
from datetime import datetime
dt = datetime.now()
res = dt.strftime("%Y-%m-%d-%H-%M")
print(res, type(res))

In [ ]:
# 时间戳 -> datetime类型?
from datetime import datetime
import time
ctime = time.localtime()
print(ctime)
res = time.strftime("%Y--%m--%d %H:%M:%S", ctime)
print(res, type(res))

In [ ]:
queryset = UserInfo.objects.all()
# 内容获取知识点
for obj in queryset:
# 1. datetime转换 
    obj.creat_time.strftime("%Y-%m-%d")
# 2. choices 获取
    obj.gender # -> 1\2
    obj.get_gender_display() # ->1对应男，2对应女 get_属性名_display()
# 3. 连表查询
    obj.depart_id # -> id数据
    obj.depart.title # -> id对应的表中title

In [ ]:
# 初识Django Form组件
# 1. views.py
class MyForm(Django中的forms):
    user = forms.CharField(widget=forms.Input)
    pwd = forms.CharField(widget=forms.Input)
    email = forms.CharField(widget=forms.Input)

def user_add(request):
    if request.method == "GET":
        form = MyForm()
        return render(request, "user_add.html", {"form": form})



# 2. user_add.html
<form method="post">
    {% csrf_token %}
    # <div class="form-group">
    #     <label>部门名称</label>
    #     <input type="text" class="form-control" placeholder="部门名称" name="title">
    # </div>
    {% for field in form %}
        {{ field }} # 一个field相当于一个form.属性名
    {% endfor %}

    # 不使用for循环
    {% form.user %}
    {% form.pwd %}
    {% form.email %}
</form>

In [ ]:
# ModelsForm组件 （推荐做法）
# 1. 在原有的Form基础上进一步简化代码
# 1.1 views.py
class MyForm(forms.ModelForm):
    # xx = forms.CharField(.....) 支持models基础上自定义的内容
    # 1.2 字段校验，保存库时候，校验
    # from django.core.validators import RegexValidator
    # mobile = forms.CharField(label="手机号",
    #     validator=[RegexValidator(r'^1\d{10}$', '手机号1开头，11位数'), ]
    # )
    class Meta:
        model = UserInfo # 实例话一个models的实例
        fields = ["name", "password", ....] # 加入想要显示的字段名称
        # fields = "__all__" 加入所有字段来显示
        # exclude = ["password"] 排除list内字段显示剩余字段

def user_add(request):
    if request.method == "GET":
        form = MyForm()
        return render(request, "user_add.html", {"form": form})

In [ ]:
# models.py 内，类方法加入__str__方法，返回想要显示的内容
# 保证在需要用的的时候，显示正确信息，而不是一个obj
class Department(models.Model):
    ''' 部门表 '''
    title = models.CharField(verbose_name='标题', max_length=32)
    docs = models.CharField(verbose_name="描述", max_length=100, default="描述")

    def __str__(self) -> str:
        return self.title

In [ ]:
# ModelForm自动填充
def user_edit(request, nid):
    row_obj = UserInfo.objects.filter(id=nid).first()
    form = UserModelForm(instance=row_obj)
    return render(request, 'user_edit.html', {"form": form})

In [ ]:
row_obj = UserInfo.objects.filter(id=nid).first()
# 这里用ModelForm更新表，需要加入更新对象，不然就是新增
form = UserModelForm(data=request.POST, instance=row_obj)
if form.is_valid():
    form.save()
    return redirect('/user/list/')

In [ ]:
# form.save的补充知识
form.sava() 实际是保存输入的所有内容，如果在输入内容上存在其他要保存到数据库中的数据可以用下面的方法新增
from.instance.字段名 = 值
form.save()

In [ ]:
# ModelForm组件，关于字段验证
# 1. views.py
class MyForm(forms.ModelForm):
    # 方法一
    # 1.1 字段校验，保存库时候，校验 
    # from django.core.validators import RegexValidator
    # mobile = forms.CharField(label="手机号",
    #     validator=[RegexValidator(r'^1\d{10}$', '手机号1开头，11位数'), ]
    # )
    class Meta:
        model = UserInfo # 实例话一个models的实例
        fields = ["name", "password", ....] # 加入想要显示的字段名称
        # fields = "__all__" 加入所有字段来显示
        # exclude = ["password"] 排除list内字段显示剩余字段

    # 方法二 : 钩子函数，clean_字段名称
    def clean_name(self):
        txt_name = self.cleaned_data['name'] # 用户输入的该字段内容
        pass # 这里写 对于name字段自己想要的验证规则
        # 验证成功，直接返回用户输入
        # return txt_name
        # 验证失败
        from django.core.exceptions import ValidationError
        return ValidationError("姓名重复")

In [ ]:
# ****************** PhoneEditModelForm ******************
class PhoneEditModelForm(forms.ModelForm):
    mobile = forms.CharField(disabled=True)  # 显示但是不可编辑

    class Meta:
        model = PhoneNumber
        # fields = ["mobile", "price", "level", "status"]
        fields = "__all__"

    # 这里统一为展示的标签增加class属性，保留统一的样式
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        for name, field in self.fields.items():
            # if name == 'password':
            #     field.widget.attrs = {"type": "password"}
            # elif name == "creat_time":
            #     field.widget.attrs = {"type": "Date"}
            field.widget.attrs = {"class": "form-control"}
    
    # 钩子方法：在编辑时，排除自身去验证字段重复
    def clean_mobile(self):
        # 当前编辑的那一行的ID ，其实就是该对象的主键pk。
        # 因为实例化该类时，传入了instance，代表的row_obj对象
        pk_id = self.instance.pk
        txt_mobile = self.cleaned_data['mobile']
        # 判断是当前修改ID，但是号码相同的数据，是否存在
        exist_mobile = PhoneNumber.objects.exclude(id=pk_id).filter(mobile=txt_mobile).exists()
        if exist_mobile:
            raise ValidationError("手机号已经存在")
        return txt_mobile
# ****************** end PhoneEditModelForm ******************

In [ ]:
# 认识filter的多条件查询
# 1.1
models.PhoneNumber.objects.filter(mobile='12588888888', id=2)

# 1.2
data_dict = {
    "mobile": "12588888888",
    "id": 2
}
models.PhoneNumber.objects.filter(**data_dict)

In [ ]:
# Django数字的筛选
models.PhoneNumber.objects.filter(id=2) # 等于2
models.PhoneNumber.objects.filter(id__gt=2) # 大于，双下划线
models.PhoneNumber.objects.filter(id__gte=2) # 大于等于，双下划线
models.PhoneNumber.objects.filter(id__lt=2) # 小于，双下划线
models.PhoneNumber.objects.filter(id__lte=2) # 小于等于，双下划线

In [ ]:
# Django字符串的筛选
models.PhoneNumber.objects.filter(mobile="123") # 完全等于
models.PhoneNumber.objects.filter(mobile__startwith="123") # 以xxx开始
models.PhoneNumber.objects.filter(mobile__endwith="123") # 以xxx结尾
models.PhoneNumber.objects.filter(mobile__contains="123") # 包含xxx

In [ ]:
# 分页
models.PhoneNumber.objects.all() # 所有数据
models.PhoneNumber.objects.all()[0:10] # 所有数据前10条

models.PhoneNumber.objects.filter(id=2) # 符合筛选条件的数据
models.PhoneNumber.objects.filter(id=2)[0:10] # 符合筛选条件的前10条数据

# 第一页
models.PhoneNumber.objects.all()[0:10] # 所有数据前10条
# 第二页
models.PhoneNumber.objects.all()[10:20] # 所有数据前20条
# 第三页
models.PhoneNumber.objects.all()[20:30] # 所有数据前30条

# 切片规则
[("当前页码" - 1) * 10, "当前页码" * 10]

# 获取数据总条数
models.PhoneNumber.objects.all().count()

models.PhoneNumber.objects.filter(id=2).count()

In [ ]:
# 分页组件
'''
 * @Descripttion : 
 * @Author       : Tommy
 * @Date         : 2022-06-14 15:59:12
 * @LastEditors  : Tommy
 * @LastEditTime : 2022-06-14 16:32:19
'''
from django.utils.safestring import mark_safe


class PageNation():
    """ 自定义分页组件 """

    def __init__(self, request, queryset, page_size=15) -> None:
        self.page = int(request.GET.get("page", 1))
        self.page_size = page_size
        self.start = (self.page - 1) * 15
        self.end = self.page * 15
        self.page_queryset = queryset[self.start:self.end]
        total_count = queryset.count()
        total_page_count, div = divmod(total_count, 15)
        if div:
            total_page_count += 1
        self.total_page_count = total_page_count

    def html(self):
        # 数据库中，数据分页小于11页
        if self.total_page_count <= 11:
            start_page = 1
            end_page = self.total_page_count
        else:
            # 小极值的处理
            if self.page <= 5:
                start_page = 1
                end_page = 11
            else:
                # 大极值的处理
                if self.page + 5 > self.total_page_count:
                    start_page = self.total_page_count - 10
                    end_page = self.total_page_count
                else:
                    start_page = self.page - 5
                    end_page = self.page + 5

        page_str_list = []
        # 首页
        page_str_list.append('<li><a href="?page={}">首页</a></li>'.format(1))
        # 上一页
        if self.page > 1:
            prev = '<li><a href="?page={}">上一页</a></li>'.format(self.page - 1)
        else:
            prev = '<li><a href="?page={}">上一页</a></li>'.format(1)
        page_str_list.append(prev)
        for i in range(start_page, end_page + 1):
            if i != self.page:
                ele = '<li><a href="?page={}">{}</a></li>'.format(i, i)
            else:
                ele = '<li class="active"><a href="?page={}">{}</a></li>'.format(
                    i, i)
            page_str_list.append(ele)


        # 下一页
        if self.page < self.total_page_count:
            prev = '<li><a href="?page={}">下一页</a></li>'.format(self.page + 1)
        else:
            prev = '<li><a href="?page={}">下一页</a></li>'.format(
                self.total_page_count)
        page_str_list.append(prev)

        # 尾页
        page_str_list.append('<li><a href="?page={}">尾页</a></li>'.format(
            self.total_page_count))

        serch_string = '''
        <div style="float: right; width:100px;">
            <form method='get'>
                <div class="input-group">
                    <input type="text" name="page" class="form-control" placeholder="跳转">
                        <span class="input-group-btn">
                        <button class="btn btn-default" type="submit">
                            go
                        </button>
                </div>
            </form>
        </div>
        '''
        page_str_list.append(serch_string)

        return mark_safe("".join(page_str_list))


In [ ]:
# 分页组件bug，分页 + 搜索
# 搜索的url拼接携带q的参数
http://localhost:8000/phone/list/?q=22
# 点击分页后，q的参数被page覆盖
http://localhost:8000/phone/list/?page=81
# 所以我们应到保留q的参数情况，存在q的时候，拼接page参数
http://localhost:8000/phone/list/?q=22&page=81
# request
print(request.GET) # -> <QueryDict: {'q': ['22'], 'page': ['81']}>
print(request.GET.urlencode()) # -> q=22&page=82

# 但是拼接请求，django默认是不允许修改QueryDict的
# 可以尝试复制一份出来
import copy
query_dict = copy.deepcopy(request.GET)
query_dict._mutable = True # 修改参数，变为可拼接的querydict
query_dict.setlist('page', [11]) # setlist,拼接page参数
query_dict.urlencode() # 拼接好后的url参数

In [ ]:
# 重构pageNation分页组件
'''
 * @Descripttion : 
 * @Author       : Tommy
 * @Date         : 2022-06-14 15:59:12
 * @LastEditors  : Tommy
 * @LastEditTime : 2022-06-14 17:37:40
'''
'''
视图函数中
    1.在views中，根据自己的情况获得数据
    queryset = PhoneNumber.objects.filter(**data_dict).order_by("level")
    2.实例化分页组件对象
    pageNationObject = PageNation(request, queryset)
    3.返回前端页面数据
    return render(request, 'phone_list.html', {
            "page_queryset": pageNationObject.page_queryset,
            "value": value,
            "page_string": pageNationObject.html()
        })

前端
    {% for obj in page_queryset%}
        ...
    {% endfor %}
    <ul class="pagination" style="float: right;">
        {{ page_string }}     
    </ul>
'''

import copy
from django.utils.safestring import mark_safe


class PageNation():
    """ 自定义分页组件 """

    def __init__(self, request, queryset, page_size=15) -> None:
        self.page = int(request.GET.get("page", 1))
        self.page_size = page_size
        self.start = (self.page - 1) * 15
        self.end = self.page * 15
        self.page_queryset = queryset[self.start:self.end]
        total_count = queryset.count()
        total_page_count, div = divmod(total_count, 15)
        if div:
            total_page_count += 1
        self.total_page_count = total_page_count
        self.query_dict = copy.deepcopy(request.GET)
        self.query_dict._mutable = True  # 修改参数，变为可拼接的querydict

    def html(self):
        # 数据库中，数据分页小于11页
        if self.total_page_count <= 11:
            start_page = 1
            end_page = self.total_page_count
        else:
            # 小极值的处理
            if self.page <= 5:
                start_page = 1
                end_page = 11
            else:
                # 大极值的处理
                if self.page + 5 > self.total_page_count:
                    start_page = self.total_page_count - 10
                    end_page = self.total_page_count
                else:
                    start_page = self.page - 5
                    end_page = self.page + 5

        page_str_list = []
        # 首页
        self.query_dict.setlist('page', [1])
        page_str_list.append('<li><a href="?{}">首页</a></li>'.format(
            self.query_dict.urlencode()))
        # 上一页
        if self.page > 1:
            self.query_dict.setlist('page', [self.page - 1])
            prev = '<li><a href="?{}">上一页</a></li>'.format(
                self.query_dict.urlencode())
        else:
            self.query_dict.setlist('page', [1])
            prev = '<li><a href="?{}">上一页</a></li>'.format(
                self.query_dict.urlencode())

        page_str_list.append(prev)
        for i in range(start_page, end_page + 1):
            self.query_dict.setlist('page', [i])
            if i != self.page:
                ele = '<li><a href="?{}">{}</a></li>'.format(
                    self.query_dict.urlencode(), i)
            else:
                ele = '<li class="active"><a href="?{}">{}</a></li>'.format(
                    self.query_dict.urlencode(), i)
            page_str_list.append(ele)

        # 下一页
        if self.page < self.total_page_count:
            self.query_dict.setlist('page', [self.page + 1])
            prev = '<li><a href="?{}">下一页</a></li>'.format(
                self.query_dict.urlencode())
        else:
            self.query_dict.setlist('page', [self.total_page_count])
            prev = '<li><a href="?{}">下一页</a></li>'.format(
                self.query_dict.urlencode())
        page_str_list.append(prev)

        # 尾页
        self.query_dict.setlist('page', [self.total_page_count])
        page_str_list.append('<li><a href="?{}">尾页</a></li>'.format(self.query_dict.urlencode()))

        serch_string = '''
        <div style="float: right; width:100px;">
            <form method='get'>
                <div class="input-group">
                    <input type="text" name="page" class="form-control" placeholder="跳转">
                        <span class="input-group-btn">
                        <button class="btn btn-default" type="submit">
                            go
                        </button>
                </div>
            </form>
        </div>
        '''
        page_str_list.append(serch_string)

        return mark_safe("".join(page_str_list))


In [ ]:
# Django中间件
# 1.1 定义一个类
# - 中间件实际是一个类，自定义的中间件就是定义一个类
# 2.1 settings.py 注册我们定义的类
MIDDLEWARE = [
    ....,
    'app.dirname.filename.className'
]

In [ ]:
# 认识Ajax
# 发送请求时，post和get，特点是：页面整体刷新
# 基于Ajax，向后台请求，局部刷新
# - 依赖JQuery
# - 编写Ajax代码
$.ajax({
    url: "发送的地址",
    type: "请求方式，post\get",
    data: {
        parm1: 111,
        parm2: 222
    },
    success:function(res){
        console.log(res);
    }
})

In [ ]:
from django.forms.utils import ErrorDict